In [2]:
import pandas as pd
from datetime import datetime

### Transformacion

In [ ]:
datos=pd.read_csv("Scraping_Data_total.csv")
datos.head()

In [ ]:
##Funciones hora modifican el formato de las horas extraidas a treves del web scraping
def hora_latam(x):
    """
    This function converts a time string in Latin American format to a 24-hour format.
    
    :param x: The input string containing the time in Latin American format, for example "3:30 p.m." or
    "10:45 a.m."
    :return: a string representing the time in 24-hour format (HH:MM) based on the input string 'x'
    which represents a time in Latin America format (e.g. "3 p.m."). If the input string is not in the
    expected format, the function returns a string representing the time in 24-hour format assuming it
    is in the format '%I:%M:%S
    """
    try:
        l=x.split()
        if l[1]=='p.':
            hora=datetime.strptime(l[0]+" PM", '%I:%M %p').time()
        elif l[1] =='a.':
            hora=datetime.strptime(l[0]+" AM", '%I:%M %p').time()
    except Exception as e:
        hora=datetime.strptime(l[0]+" AM", '%I:%M:%S %p').time()
    return hora.strftime('%H:%M')


def hora_wingo(x):
    """
    This function takes a string representing a time in either 12-hour or 24-hour format and returns the
    time in 24-hour format.
    
    :param x: The input parameter to the function hora_wingo, which is expected to be a string
    representing a time in either 12-hour or 24-hour format
    :return: a string in the format '%H:%M', which represents the time in 24-hour format.
    """
    try:
        x=x.split()
        if len(x)==3:
            hora=datetime.strptime(x[1]+" "+x[2], '%I:%M %p').time()
        elif len(x)==2:
            if x[1]=='a.m':
                hora=datetime.strptime(x[0]+" AM", '%I:%M %p').time()
            elif x[1]=='p.m':
                hora=datetime.strptime(x[0]+" PM", '%I:%M %p').time()
    except Exception as e:
        hora=datetime.strptime(x[0]+" PM", '%I:%M:%S %p').time()
        #print(e)
    return hora.strftime('%H:%M')

#como usar
#datos['Hora_Salida']=datos.apply(lambda x: hora_latam(x.Hora_Salida) if x.Aerolinea =='Latam' else hora_wingo(x.Hora_Salida), axis=1)
#datos['Hora_LLegada']=datos.apply(lambda x: hora_latam(x.Hora_LLegada) if x.Aerolinea =='Latam' else hora_wingo(x.Hora_LLegada), axis=1)

In [ ]:
datos['Price']=datos['Price'].str.replace(",", ".")
datos['Price']=datos['Price'].str.replace("$", " ")
datos['Price']=datos['Price'].str.replace("COP",'')
datos['Price']=datos['Price'].str.replace(" ",'')

In [ ]:
##funcion para modifiar el precio
def Precio(x):
    """
    The function takes a string representing a price and returns an integer value without the decimal
    point.
    
    :param x: The input parameter x is a string representing a price value in the format of
    "dollars.cents". For example, "10.99" represents a price of ten dollars and ninety-nine cents
    :return: an integer value that represents the price of a product. The function takes a string as
    input, which represents the price in a certain format (e.g. "10.99"). The function then converts
    this string into an integer value by removing the decimal point and returning the integer value.
    """
    l=x.split(".")
    if len(l)==1:
        return int(l[0])
    else:
        l1=l[:-1]
        num=''.join(l1)
        return int(num)

datos['Price']=datos['Price'].apply(lambda x: Precio(x))

### Limpieza

In [3]:
#leer el archivo con los datos
Data=pd.read_csv("../Data/Scraping_Data_total_2.csv")

In [4]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7062 entries, 0 to 7061
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CaptureDate   7062 non-null   object
 1   Aerolinea     7062 non-null   object
 2   Fecha         7062 non-null   object
 3   Departure     7062 non-null   object
 4   Arrive        7062 non-null   object
 5   Hora_Salida   7062 non-null   object
 6   Hora_LLegada  7062 non-null   object
 7   Price         7062 non-null   int64 
 8   Tipo          7062 non-null   object
dtypes: int64(1), object(8)
memory usage: 496.7+ KB


In [5]:
#verificamos si exixte algun campo nulo
Data.isnull().sum()

CaptureDate     0
Aerolinea       0
Fecha           0
Departure       0
Arrive          0
Hora_Salida     0
Hora_LLegada    0
Price           0
Tipo            0
dtype: int64

In [6]:
#verificamos si exisiten datos duplicados
Data[Data.duplicated()]

,CaptureDate,Aerolinea,Fecha,Departure,Arrive,Hora_Salida,Hora_LLegada,Price,Tipo
2103,2023-01-23,Latam,27/01/2023,BOG,CLO,10:00,11:11,266130,Directo
2247,2023-01-23,Latam,3/02/2023,BOG,CLO,10:00,11:11,266130,Directo
2370,2023-01-23,Latam,10/02/2023,BOG,CLO,10:00,11:11,266130,Directo
2515,2023-01-23,Latam,17/02/2023,BOG,CLO,10:00,11:11,266130,Directo
5840,2023-04-06,Latam,11/05/2023,MDE,BOG,04:04,05:00,258260,Directo
5902,2023-04-06,Latam,15/05/2023,MDE,BOG,04:04,05:00,258260,Directo
5963,2023-04-06,Latam,18/05/2023,MDE,BOG,04:04,05:00,155210,Directo


In [7]:
#Eliminar las datos duplicados
Data_sin_duplicados = Data.drop_duplicates().copy()

In [16]:
#Data_sin_duplicados['Fecha']=pd.to_datetime(Data_sin_duplicados['Fecha'])
#Data_sin_duplicados['CaptureDate']=pd.to_datetime(Data_sin_duplicados['CaptureDate'])


In [8]:
#se convirten las fechas a un formato adaptable
def fecha(x):
    try:
        return datetime.strptime(x, '%d/%m/%Y')
    except:
        return datetime.strptime(x, '%Y-%m-%d')
Data_sin_duplicados['Fecha']=Data_sin_duplicados['Fecha'].apply(lambda x: fecha(x))
Data_sin_duplicados['CaptureDate']=Data_sin_duplicados['CaptureDate'].apply(lambda x: fecha(x))


In [9]:
Data_sin_duplicados['Hora_Salida'] = pd.to_datetime(Data_sin_duplicados['Hora_Salida'], format='%H:%M').dt.time
Data_sin_duplicados['Hora_LLegada'] = pd.to_datetime(Data_sin_duplicados['Hora_LLegada'], format='%H:%M').dt.time

In [10]:
Data_sin_duplicados['Tipo']=Data_sin_duplicados['Tipo'].str.replace(" ",'_')
Data_sin_duplicados['Tipo']=Data_sin_duplicados['Tipo'].str.replace("*",'')
Data_sin_duplicados['Tipo']=Data_sin_duplicados['Tipo'].str.replace(" ",'')
Data_sin_duplicados['Tipo'].unique()

C:\Users\DELL\AppData\Local\Temp\ipykernel_20136\1468271064.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Data_sin_duplicados['Tipo']=Data_sin_duplicados['Tipo'].str.replace("*",'')


array(['Directo', '1_parada', '2_paradas'], dtype=object)

In [11]:
Data_sin_duplicados

,CaptureDate,Aerolinea,Fecha,Departure,Arrive,Hora_Salida,Hora_LLegada,Price,Tipo
0,2023-04-18,Latam,2023-04-23,SMR,MDE,20:35:00,21:51:00,993760,Directo
1,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,10:44:00,214790,1_parada
2,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,11:11:00,214790,1_parada
3,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,12:56:00,214790,1_parada
4,2023-04-18,Latam,2023-04-23,SMR,MDE,07:43:00,11:11:00,227280,1_parada
...,...,...,...,...,...,...,...,...,...
7057,2023-04-17,Latam,2023-05-24,MDE,BOG,17:27:00,20:57:00,278020,1_parada
7058,2023-04-17,Latam,2023-05-24,MDE,BOG,17:27:00,22:13:00,278020,1_parada
7059,2023-04-17,Latam,2023-05-24,MDE,BOG,17:27:00,22:46:00,278020,1_parada
7060,2023-04-17,Latam,2023-05-24,MDE,BOG,18:50:00,23:00:00,373450,1_parada


In [12]:
Data_sin_duplicados.to_csv("../Data/Data_Limpia.csv",index=False)